In [1]:
# Import pandas, requests library and json.
import requests
import json
import pandas as pd
import glob
from pathlib import Path


In [2]:
# Create list of air quality CSVs.
csv_file_list = ["daily_aqi_by_county_2009.csv", "daily_aqi_by_county_2010.csv", "daily_aqi_by_county_2011.csv", "daily_aqi_by_county_2012.csv", "daily_aqi_by_county_2013.csv", "daily_aqi_by_county_2014.csv", "daily_aqi_by_county_2015.csv", "daily_aqi_by_county_2016.csv", "daily_aqi_by_county_2017.csv", "daily_aqi_by_county_2018.csv", "daily_aqi_by_county_2019.csv"]


In [3]:
# Create path for CSVs
data_folder = Path("./Resources/")

In [4]:
# Create DataFrames from CSV list and merge into single DataFrame using concat.
list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(data_folder/(filename)))

merged_df = pd.concat(list_of_dataframes)

print(merged_df)

       State Name county Name  State Code  County Code        Date  AQI  \
0         Alabama     Baldwin           1            3  2009-03-05   56   
1         Alabama     Baldwin           1            3  2009-03-08   18   
2         Alabama     Baldwin           1            3  2009-03-10   31   
3         Alabama     Baldwin           1            3  2009-03-11   28   
4         Alabama     Baldwin           1            3  2009-03-12   30   
...           ...         ...         ...          ...         ...  ...   
340164    Wyoming      Weston          56           45  2019-12-27   36   
340165    Wyoming      Weston          56           45  2019-12-28   37   
340166    Wyoming      Weston          56           45  2019-12-29   34   
340167    Wyoming      Weston          56           45  2019-12-30   36   
340168    Wyoming      Weston          56           45  2019-12-31   40   

        Category Defining Parameter Defining Site  Number of Sites Reporting  
0       Moderate    

In [5]:
# Rename columns in DataFrame

merged_df.rename(columns={
    'State Name': 'state_name', 
    'county Name': 'county_name', 
    'State Code': 'state_code', 
    'County Code': 'county_code',
    'Date': 'date',
    'AQI': 'aqi',
    'Category': 'category',
    'Defining Parameter': 'defining_parameter',
    'Defining Site': 'defining_site',
    'Number of Sites Reporting': 'reporting_sites'}, inplace=True)


In [6]:
# Filter DataFrame for California Data

ca_df = merged_df.loc[(merged_df.state_name == "California")]

In [7]:
print(ca_df)

       state_name county_name  state_code  county_code        date  aqi  \
14871  California     Alameda           6            1  2009-01-01   84   
14872  California     Alameda           6            1  2009-01-02   60   
14873  California     Alameda           6            1  2009-01-03   54   
14874  California     Alameda           6            1  2009-01-04   56   
14875  California     Alameda           6            1  2009-01-05   55   
...           ...         ...         ...          ...         ...  ...   
34174  California        Yolo           6          113  2019-12-27   29   
34175  California        Yolo           6          113  2019-12-28   46   
34176  California        Yolo           6          113  2019-12-29   67   
34177  California        Yolo           6          113  2019-12-30   28   
34178  California        Yolo           6          113  2019-12-31   52   

       category defining_parameter defining_site  reporting_sites  
14871  Moderate              PM

In [8]:
# Renaming California DataFrame
air_quality_df = pd.DataFrame(ca_df)

In [9]:
air_quality_df.head()

,state_name,county_name,state_code,county_code,date,aqi,category,defining_parameter,defining_site,reporting_sites
14871,California,Alameda,6,1,2009-01-01,84,Moderate,PM2.5,06-001-2004,4
14872,California,Alameda,6,1,2009-01-02,60,Moderate,PM2.5,06-001-2004,4
14873,California,Alameda,6,1,2009-01-03,54,Moderate,PM2.5,06-001-2004,4
14874,California,Alameda,6,1,2009-01-04,56,Moderate,PM2.5,06-001-2004,4
14875,California,Alameda,6,1,2009-01-05,55,Moderate,PM2.5,06-001-0007,4


In [10]:
# Check column types
air_quality_df.dtypes

state_name            object
county_name           object
state_code             int64
county_code            int64
date                  object
aqi                    int64
category              object
defining_parameter    object
defining_site         object
reporting_sites        int64
dtype: object

In [11]:
# Drop unnecessary columns
air_quality_df = air_quality_df.drop(['state_name', 'state_code', 'defining_site', 'reporting_sites'], axis=1)

In [12]:
# Drop na air quality values for county name and aqi
aq_cleaned_df = air_quality_df.dropna(subset=['county_name', 'aqi'])

In [13]:
aq_cleaned_df

,county_name,county_code,date,aqi,category,defining_parameter
14871,Alameda,1,2009-01-01,84,Moderate,PM2.5
14872,Alameda,1,2009-01-02,60,Moderate,PM2.5
14873,Alameda,1,2009-01-03,54,Moderate,PM2.5
14874,Alameda,1,2009-01-04,56,Moderate,PM2.5
14875,Alameda,1,2009-01-05,55,Moderate,PM2.5
...,...,...,...,...,...,...
34174,Yolo,113,2019-12-27,29,Good,Ozone
34175,Yolo,113,2019-12-28,46,Good,PM2.5
34176,Yolo,113,2019-12-29,67,Moderate,PM2.5
34177,Yolo,113,2019-12-30,28,Good,PM2.5


In [14]:
# Filter dataframe for error reading records in aqi assuming reading over 400 is error
aq_filtered_df = aq_cleaned_df[aq_cleaned_df['aqi'] <= 400]
aq_filtered_df

,county_name,county_code,date,aqi,category,defining_parameter
14871,Alameda,1,2009-01-01,84,Moderate,PM2.5
14872,Alameda,1,2009-01-02,60,Moderate,PM2.5
14873,Alameda,1,2009-01-03,54,Moderate,PM2.5
14874,Alameda,1,2009-01-04,56,Moderate,PM2.5
14875,Alameda,1,2009-01-05,55,Moderate,PM2.5
...,...,...,...,...,...,...
34174,Yolo,113,2019-12-27,29,Good,Ozone
34175,Yolo,113,2019-12-28,46,Good,PM2.5
34176,Yolo,113,2019-12-29,67,Moderate,PM2.5
34177,Yolo,113,2019-12-30,28,Good,PM2.5


In [15]:
# Export to csv
aq_cleaned_df.to_csv(r'aq_dataframe.csv', index = False)
